In [26]:
from pydantic import BaseModel, Field
from typing import List, Optional

class Question(BaseModel):
    topic: str = Field(description="Topic of the questions that has been generated")
    question: str = Field(description="Interview question for the candidate")
    answer: str = Field(description="Suggested answer for the question")

In [27]:
"""State management for the index graph."""
from __future__ import annotations
from dataclasses import dataclass, field
from typing import Annotated, Optional

from langchain_core.documents import Document

from dataclasses import dataclass, field
from typing import Optional, TypedDict, List, Dict, Union, Annotated
from langchain.schema import AgentAction, AgentFinish
import operator


class AgentState(TypedDict):

    resume: str = field(
        metadata={
            "description": "The resume of the candidate."
        }
    )
    job_description: str = field(
        metadata={
            "description": "The job description for the role, represented as a string."
        }
    )
    job_title: str = field(
        metadata={
            "description": "The job title for which questions and evaluations are generated."
        }
    )
    company_name: str = field(
        metadata={
            "description": "The name of the company associated with the job description."
        }
    )
    history: Optional[List[Dict[str, str]]] = field(
        default=None,
        metadata={
            "description": "History of previously asked questions and answers as a list of dictionaries."
        }
    )
    agent_out: Union[AgentAction, AgentFinish, None] = field(
        default=None,
        metadata={
            "description": "The output from the agent, which can be an action or a finish signal."
        }
    )
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add] = field(
        default_factory=list,
        metadata={
            "description": "A list of intermediate steps taken by the agent, each represented as a tuple of action and description."
        }
    )

In [29]:
"""This "graph" simply exposes an endpoint for a user to upload docs to be indexed."""

import json
import os
from typing import Optional
from langchain_core.runnables import RunnableConfig
from langgraph.graph import END, START, StateGraph
from langchain_core.tools import tool
from langchain_openai.chat_models import AzureChatOpenAI
from langsmith import Client
from dotenv import load_dotenv
import asyncio
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
from http.client import HTTPException
import itertools
import random
import uuid
import json

import certifi
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_community.chat_models import ChatAnyscale, BedrockChat
from langchain_openai.chat_models import AzureChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, PromptTemplate, HumanMessagePromptTemplate
from dotenv import load_dotenv
from langchain_core.runnables import RunnableParallel
import os
from urllib3.util import parse_url

load_dotenv()

client = Client()

model = AzureChatOpenAI(temperature=0,
                            azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
                            openai_api_version=os.getenv('AZURE_OPENAI_API_VERSION'),
                            openai_api_key=os.getenv('OPENAI_API_KEY'),
                            deployment_name=os.getenv('AZURE_OPENAI_CHATGPT_DEPLOYMENT'),
                            openai_api_type="azure"
                            )
def generate_questions(state: list): 
    """Generate interview questions using an LLM.
    Args:
        state (dict): The input state containing required fields.
    Returns:
        dict: A dictionary containing generated questions.
    """
    try:
        print("reached here")
        # Ensure the state contains all required fields
        required_fields = ["resume", "job_description", "job_title", "company_name", "history"]
        for field in required_fields:
            if field not in state:
                raise ValueError(f"Missing required field: {field}")

        document_prompt = client.pull_prompt("interviewer_prompt")
        print (document_prompt)
        print(state["company_name"])
        chain = (document_prompt | model.with_structured_output(Question))
        response = chain.invoke({
            "resume": str(state["resume"]), 
            "job_description": str(state["job_description"]),
            "job_title": str(state["job_title"]),
            "company_name": str(state["company_name"])
        }) 

        print(response)
        
        return {"agent_out": response}
    except Exception as e:
        print("Error:", e)
        return {"agent_out": f"Could not generate questions. Details: {str(e)}"}
@tool
def evaluate_answers(state: AgentState):
    """Generate interview questions using an LLM.
    Args:
        resume (str): The candidate's resume.
        job_description (str): The job description for the role.
        job_title (str): The job title of the role
        company_name (str): The name of the company
        history Optional[str]: History of previously asked questions and answers
    Returns:
        dict: A dictionary containing generated questions categorized into technical, personal, HR, and logical sections.
    """
    try:
        document_prompt = client.pull_prompt("quiz_from_jd")
        chain = (document_prompt | model.with_structured_output(Question))
        response = chain.invoke({"resume_text": state.resume, 
                           "job_description": state.job_description,
                           "job_title": state.job_title,
                           "company_name": state.company_name,
                           "history": state.history})
        return response
    except Exception as e:
        print("Error:", e)
        return {"error": f"Could not generate questions. Details: {str(e)}"}
@tool
   
def evaluate_candidate(state: AgentState):
    """Generate interview questions using an LLM.
    Args:
        resume (str): The candidate's resume.
        job_description (str): The job description for the role.
        job_title (str): The job title of the role
        company_name (str): The name of the company
        history Optional[str]: History of previously asked questions and answers
    Returns:
        dict: A dictionary containing generated questions categorized into technical, personal, HR, and logical sections.
    """
    try:
        document_prompt = client.pull_prompt("quiz_from_jd")
        chain = (document_prompt | model.with_structured_output(Question))
        response = chain.invoke({"resume_text": state.resume, 
                           "job_description": state.job_description,
                           "job_title": state.job_title,
                           "company_name": state.company_name,
                           "history": state.history})
        return response
    except Exception as e:
        print("Error:", e)
        return {"error": f"Could not generate questions. Details: {str(e)}"}
    
def count_questions(state: AgentState):
    """
    Determines whether to finish.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

# Define the graph
builder = StateGraph(AgentState)
builder.add_node(generate_questions)
builder.add_edge(START, "generate_questions")
#builder.add_edge("generate_questions", "evaluate_answers")
#builder.add_edge("evaluate_answers", "generate_questions")
builder.add_edge("generate_questions", END)
# Compile into a graph object that you can invoke and deploy.
graph = builder.compile()
graph.name = "IndexGraph"


agent_state = AgentState(
    resume="John Doe's resume text here...",
    job_description="Job description for a data scientist role...",
    job_title="Data Scientist",
    company_name="Tech Corp",
    history=[{"question": "What are your strengths?", "answer": "Problem-solving and teamwork"}]
)

result = graph.invoke({
    "resume": """Name: Rahul Krishnamoorthy
Email: rahul.krish@example.com
Phone: +1-234-567-8901
LinkedIn: linkedin.com/in/rahulkrish
GitHub: github.com/rahulkrish

Professional Summary
Dynamic AI Engineer and Full-Stack Developer with 5+ years of experience designing, developing, and deploying innovative software solutions. Proficient in backend development, API integration, and building data pipelines. Skilled in integrating GPT models and deploying CI/CD pipelines using AWS services.

Technical Skills
Programming Languages: Python, JavaScript, Java
Frameworks & Tools: FastAPI, Flask, Vue.js, LangChain
Cloud Technologies: AWS (EKS, S3, SES, MSK)
Databases: MongoDB, PostgreSQL
Other Skills: Kafka, CI/CD pipelines, Generative AI Integration
Experience
AI Engineer | Oros Tech LLC (Remote)
Jan 2022 – Present

Developed over 50+ RESTful APIs using FastAPI and Flask.
Built streaming pipelines using Kafka for real-time data processing.
Integrated GPT-4 with LangChain to create intelligent question-generation systems for recruitment.
Deployed applications on AWS infrastructure using Docker and Kubernetes.
Service Engineer | 247.ai
Jun 2019 – Dec 2021

Collaborated with clients to troubleshoot software issues and query databases for diagnostics.
Documented and resolved over 500 technical support tickets with a 95% satisfaction rate.
Education
Master of Science in Artificial Intelligence
University of California, Berkeley | 2019

Bachelor of Technology in Computer Science
Indian Institute of Technology, Madras | 2017""",
    "job_description": """Die XXXLutz-Unternehmensgruppe betreibt über 370 Einrichtungshäuser in 14 europäischen Ländern und beschäftigt mehr als 27.100 Mitarbeiter. Mit einem Jahresumsatz von 6 Milliarden Euro gehört XXXLutz zu den größten Möbelhändlern der Welt. Zudem werden bereits 24 Onlineshops in den Vertriebsschienen XXXLutz, Möbelix und Mömax betrieben. Die Zentrale des 1945 gegründeten Familienunternehmens befindet sich in Wels.


Wir als XXXLdigital Team arbeiten gemeinsam an der Zukunft des Möbelhandels. Dabei arbeiten wir nicht nur in Teams, sondern leben sie auch – den Zusammenhalt, das Gefühl, das Lernen. Wir bringen unsere Ideen ein und gestalten proaktiv die Zukunft – und das auf allen digitalen Touchpoints. Wir sind begeistert und ständig auf der Suche nach neuen Lösungswegen. Dafür suchen wir Gleichgesinnte, die diesen Weg mit uns gehen wollen und bereit sind sich neuen Herausforderungen zu stellen.

Aufgaben:
Aufbau und Entwicklung der serviceorientierten XXXLutz AI Plattform und des dazu gehörigen CI/CD Prozesses für unsere Services
Entwicklung, Implementierung und Wartung der skalierbaren Infrastruktur für die Integration, Speicherung, Verarbeitung und Analyse von Daten auf Basis von Kubernetes und Kubeflow
Mitwirkung bei der Auswahlentscheidung der einzusetzenden Toolchain sowie deren Weiterentwicklung und Automatisierung mittels Scripting
Setup und Wartung von containerisierten Apps (Docker) und eingesetzten AI Tools sowie Implementieren von Security Konzepten für unsere AI Plattform
Mitarbeit an AI Projekten""",
    "job_title": "Data Scientist",
    "company_name": "Tech Corp",
    "history": [{"question": "What are your strengths?", "answer": "Problem-solving and teamwork"}]
})
agent_state["history"].append({"question": "What is your experience with Python?", "answer": "5 years"})
print(result)


reached here
input_variables=['company_name', 'job_description', 'job_title', 'resume'] metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'interviewer_prompt', 'lc_hub_commit_hash': 'c040d8b0df39853f6c87963500a5b9cc8b89bece8dbf48fb972f02890de39f82'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a world-class interviewer. With human-provided inputs such as resume, job title, company name, job description, and history of previous conversations, you should be able to ask insightful questions. Create a total of 30-40 questions with corresponding answers. Select 7-8 topics from below based on the resume and job description and each topic should cover 5-10 questions:\n<example>\n1. Personal Growth:\n- <example> "Can you describe a significant challenge you\'ve faced in your career and how you overcame it?" \n  - Answer: "In my previous role, I faced a major project setback due to unexpected changes in client requirements. I organized a team mee